Code for fidelity test:

In [1]:
import numpy as np
from scipy.linalg import sqrtm


def fidelity_test(sigma, rho):
    # Compute the square root of sigma
    sigma_sqrt = sqrtm(sigma)
    # Compute the product of the square root of sigma and rho
    product = sigma_sqrt @ rho @ sigma_sqrt
    # Ensure the product is Hermitian
    product = 0.5 * (product + product.conj().T)
    # Compute the eigenvalues of the product
    eigenvalues = np.linalg.eigh(product)[0]
    # Sum the square roots of the eigenvalues
    fid = np.sum(np.sqrt(np.maximum(eigenvalues, 0.0)))
    
    return fid

1. Pure states: (EDITED for updated qibo version, works well)

In [ ]:
# Check fidelity:
from qibo.quantum_info import random_density_matrix
A = random_density_matrix(2, True)
B = random_density_matrix(2, True)
fidelity(A, B)

np.float64(0.765338548785599)

In [14]:
fidelity_test(A,B)

np.float64(0.7653385487855991)

Everything works as expected. 

2. Mixed states

In [6]:
# Check fidelity with a specific example:
C = np.array([[0.179521969101107, -0.00114411514091934 - 0.0114552376473287*1j, 0.0332097326275308 - 0.113794608022202*1j, 0.00487638257412585 + 0.165342599975152*1j], [-0.00114411514091934 + 0.0114552376473287*1j, 0.275440178153448, -0.0150401114186904 + 0.0806575151900997*1j, -0.0620039054996596 + 0.16741961011259*1j], [0.0332097326275308 + 0.113794608022202*1j, -0.0150401114186904 - 0.0806575151900997*1j, 0.170626725235746, -0.0166965556759144 - 0.0440636154331044*1j], [0.00487638257412585 - 0.165342599975152*1j, -0.0620039054996596 - 0.16741961011259*1j, -0.0166965556759144 + 0.0440636154331044*1j, 0.374411127509699]])
D = np.array([[0.179521969101107 + 8.52365613707931e-18*1j, -0.00114411514091961 - 0.0114552376473287*1j, 0.033209732627531 - 0.113794608022202*1j, 0.00487638257412567 + 0.165342599975152*1j], [-0.00114411514091941 + 0.0114552376473287*1j, 0.275440178153448 + 7.22539881141835e-18*1j, -0.0150401114186903 + 0.0806575151900998*1j, -0.0620039054996599 + 0.16741961011259*1j], [0.0332097326275307 + 0.113794608022202*1j, -0.0150401114186902 - 0.0806575151900997*1j, 0.170626725235746 - 3.81010862582839e-17*1j, -0.0166965556759144 - 0.0440636154331044*1j], [0.00487638257412578 - 0.165342599975152*1j, -0.0620039054996596 - 0.16741961011259*1j, -0.0166965556759144 + 0.0440636154331044*1j, 0.374411127509698 + 1.09407727602622e-18*1j]])


In [7]:
from qibo.quantum_info import fidelity
fidelity(C,D)

np.float64(0.9999017711797442)

In [8]:
fidelity_test(C,D)

np.float64(0.9999999999998552)

In [9]:
# Qibo fidelity function for mixed states with PRECISION_TOL = 1e-9:
import numpy as np
from qibo.backends import NumpyBackend

def purity(state):
    backend = NumpyBackend()
    if len(state.shape) == 1:
        pur = backend.np.real(backend.calculate_vector_norm(state)) ** 2
    else:
        pur = backend.np.real(backend.np.trace(backend.np.matmul(state, state)))
    return float(pur)

def fidelity_corr(state, target):
    backend = NumpyBackend()
    purity_state = purity(state)
    purity_target = purity(target) 
    eigenvalues, eigenvectors = backend.calculate_eigenvectors(state)
    state = np.zeros(state.shape, dtype=complex)
    for eig, eigvec in zip(eigenvalues, backend.np.transpose(eigenvectors, (1, 0))):
        matrix = backend.np.sqrt(eig) * backend.np.outer(eigvec, backend.np.conj(eigvec))
        state = state + matrix 
        del matrix
    
    fid = state @ target @ state 

    eigenvalues, eigenvectors = backend.calculate_eigenvectors(fid)
    fid = np.zeros(state.shape, dtype=complex)
    fid = backend.cast(fid, dtype=fid.dtype)
    for eig, eigvec in zip(eigenvalues, backend.np.transpose(eigenvectors, (1, 0))):
        if backend.np.real(eig) > 1e-9: # PRECISION_TOL
            matrix = backend.np.sqrt(eig) * backend.np.outer(eigvec, backend.np.conj(eigvec))
            fid = fid + matrix
            del matrix
    fid = backend.np.real(backend.np.trace(fid)) 

    return fid 

In [10]:
fidelity_corr(C,D)

np.float64(0.9999999999998724)